In [1]:

import keras
from keras.datasets import cifar10
import os
from scipy import misc
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose, Conv2D
from keras.optimizers import Adam
import os
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
import keras.backend.tensorflow_backend as KTF

from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(3,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(SNConv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(0., 1 - pred_real)) + K.mean(K.maximum(0., 1 + pred_fake))
    g_loss = -K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train

class SNConv2D(_Conv):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(SNConv2D, self).__init__(
            rank=2,
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

        self.input_spec = InputSpec(ndim=4)
        self.Ip = 1
        self.u = self.add_weight(
            name='W_u',
            shape=(1,filters),
            initializer='random_uniform',
            trainable=False
        )

    def call(self, inputs):
        outputs = K.conv2d(
            inputs,
            self.W_bar(),
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs


    def get_config(self):
        config = super(SNConv2D, self).get_config()
        config.pop('rank')
        return config

    def W_bar(self):
        # Spectrally Normalized Weight
        W_mat = K.permute_dimensions(self.kernel, (3, 2, 0, 1)) # (h, w, i, o) => (o, i, h, w)
        W_mat = K.reshape(W_mat,[K.shape(W_mat)[0], -1]) # (o, i * h * w)

        if not self.Ip >= 1:
            raise ValueError("The number of power iterations should be positive integer")

        _u = self.u
        _v = None

        for _ in range(self.Ip):
            _v = _l2normalize(K.dot(_u, W_mat))
            _u = _l2normalize(K.dot(_v, K.transpose(W_mat)))

        sigma = K.sum(K.dot(_u,W_mat)*_v)

        K.update(self.u,K.in_train_phase(_u, self.u))
        return self.kernel / sigma

def _l2normalize(x):
    return x / K.sqrt(K.sum(K.square(x)) + K.epsilon())
set_gpu_config("0",0.5)

epochs = 50
image_size = (32,32,3)
noise_size = (2,2,32)
batch_size = 64
sample_size=10
size=32
sample_interval=200
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_of_data = x_train.shape[0]
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train/255)*2-1
x_test = (x_test/255)*2-1
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

nb_batches = int(x_train.shape[0] / batch_size)
global_step = 0
for epoch in range(epochs):
    for index in range(nb_batches):
        global_step += 1
        real_images = x_train[index * batch_size:(index + 1) * batch_size]
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
        if global_step % sample_interval == 0:
            image = generator.predict(np.random.normal(size=(sample_size * sample_size,) + noise_size))
            image = np.reshape(image, (sample_size, sample_size, size, size, 3))
            image = np.transpose(image, (0, 2, 1, 3, 4))
            image = np.reshape(image, (sample_size * size, sample_size * size, 3))

            image = 255 * (image + 1) / 2
            image = image.astype("uint8")
            if not os.path.isdir('images_sngan_cifar10'):
                os.mkdir('images_sngan_cifar10')
            misc.imsave("images_sngan_cifar10/e{0}_step{1}img.png".format(epoch,global_step), image)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 64)        73792     
__________

[0/50] [0_119] d_loss: 1.338, g_loss: 0.4321
[0/50] [0_120] d_loss: 0.7836, g_loss: 0.7288
[0/50] [0_121] d_loss: 0.5658, g_loss: 1.447
[0/50] [0_122] d_loss: 0.1746, g_loss: 0.8657
[0/50] [0_123] d_loss: 0.5129, g_loss: 2.872
[0/50] [0_124] d_loss: 0.9852, g_loss: 0.5051
[0/50] [0_125] d_loss: 0.7918, g_loss: 1.632
[0/50] [0_126] d_loss: 0.3533, g_loss: 1.032
[0/50] [0_127] d_loss: 0.2494, g_loss: 2.352
[0/50] [0_128] d_loss: 0.3628, g_loss: 1.084
[0/50] [0_129] d_loss: 0.281, g_loss: 3.769
[0/50] [0_130] d_loss: 0.7599, g_loss: 1.208
[0/50] [0_131] d_loss: 0.1902, g_loss: 0.8865
[0/50] [0_132] d_loss: 0.4407, g_loss: 2.982
[0/50] [0_133] d_loss: 0.5094, g_loss: 1.322
[0/50] [0_134] d_loss: 0.07022, g_loss: 1.005
[0/50] [0_135] d_loss: 0.2288, g_loss: 2.887
[0/50] [0_136] d_loss: 0.3968, g_loss: 1.386
[0/50] [0_137] d_loss: 0.09805, g_loss: 1.014
[0/50] [0_138] d_loss: 0.2222, g_loss: 3.967
[0/50] [0_139] d_loss: 0.4662, g_loss: 1.261
[0/50] [0_140] d_loss: 0.03318, g_loss: 2.472
[0/5

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:234: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


[0/50] [0_201] d_loss: 0.3869, g_loss: 1.227
[0/50] [0_202] d_loss: 0.6997, g_loss: 2.043
[0/50] [0_203] d_loss: 0.1711, g_loss: 1.184
[0/50] [0_204] d_loss: 0.8566, g_loss: 2.266
[0/50] [0_205] d_loss: 0.6938, g_loss: 0.769
[0/50] [0_206] d_loss: 1.122, g_loss: 1.344
[0/50] [0_207] d_loss: 0.4515, g_loss: 2.138
[0/50] [0_208] d_loss: 0.1749, g_loss: 1.13
[0/50] [0_209] d_loss: 0.9264, g_loss: 2.401
[0/50] [0_210] d_loss: 0.6118, g_loss: 1.091
[0/50] [0_211] d_loss: 0.6267, g_loss: 2.708
[0/50] [0_212] d_loss: 0.8249, g_loss: 1.055
[0/50] [0_213] d_loss: 0.5285, g_loss: 2.252
[0/50] [0_214] d_loss: 0.3983, g_loss: 1.443
[0/50] [0_215] d_loss: 0.4333, g_loss: 2.872
[0/50] [0_216] d_loss: 0.9619, g_loss: 0.9143
[0/50] [0_217] d_loss: 0.5286, g_loss: 2.445
[0/50] [0_218] d_loss: 0.2134, g_loss: 1.089
[0/50] [0_219] d_loss: 1.002, g_loss: 3.479
[0/50] [0_220] d_loss: 0.8783, g_loss: 1.135
[0/50] [0_221] d_loss: 0.3592, g_loss: 2.796
[0/50] [0_222] d_loss: 0.4963, g_loss: 1.182
[0/50] [0_22

...

[49/50] [49_38301] d_loss: 0.5805, g_loss: 1.55
[49/50] [49_38302] d_loss: 0.3591, g_loss: 2.364
[49/50] [49_38303] d_loss: 0.353, g_loss: 1.902
[49/50] [49_38304] d_loss: 0.3865, g_loss: 3.092
[49/50] [49_38305] d_loss: 0.196, g_loss: 2.221
[49/50] [49_38306] d_loss: 0.3375, g_loss: 3.431
[49/50] [49_38307] d_loss: 0.3921, g_loss: 1.233
[49/50] [49_38308] d_loss: 0.7874, g_loss: 1.819
[49/50] [49_38309] d_loss: 0.7212, g_loss: 1.687
[49/50] [49_38310] d_loss: 0.459, g_loss: 2.196
[49/50] [49_38311] d_loss: 0.5071, g_loss: 3.064
[49/50] [49_38312] d_loss: 0.5302, g_loss: 1.596
[49/50] [49_38313] d_loss: 0.4262, g_loss: 2.09
[49/50] [49_38314] d_loss: 0.6079, g_loss: 3.017
[49/50] [49_38315] d_loss: 0.7404, g_loss: 1.083
[49/50] [49_38316] d_loss: 0.5722, g_loss: 2.275
[49/50] [49_38317] d_loss: 0.2801, g_loss: 1.671
[49/50] [49_38318] d_loss: 0.5239, g_loss: 2.311
[49/50] [49_38319] d_loss: 0.4043, g_loss: 1.657
[49/50] [49_38320] d_loss: 0.3981, g_loss: 2.433
[49/50] [49_38321] d_loss

[49/50] [49_38477] d_loss: 0.7519, g_loss: 2.959
[49/50] [49_38478] d_loss: 1.05, g_loss: 0.4689
[49/50] [49_38479] d_loss: 1.211, g_loss: 1.232
[49/50] [49_38480] d_loss: 0.5411, g_loss: 2.075
[49/50] [49_38481] d_loss: 0.3818, g_loss: 2.429
[49/50] [49_38482] d_loss: 0.1983, g_loss: 2.081
[49/50] [49_38483] d_loss: 0.4411, g_loss: 1.914
[49/50] [49_38484] d_loss: 0.2483, g_loss: 1.938
[49/50] [49_38485] d_loss: 0.2961, g_loss: 2.004
[49/50] [49_38486] d_loss: 0.2946, g_loss: 2.233
[49/50] [49_38487] d_loss: 0.2652, g_loss: 2.38
[49/50] [49_38488] d_loss: 0.2191, g_loss: 2.743
[49/50] [49_38489] d_loss: 0.4492, g_loss: 1.882
[49/50] [49_38490] d_loss: 0.4964, g_loss: 2.482
[49/50] [49_38491] d_loss: 0.3906, g_loss: 2.396
[49/50] [49_38492] d_loss: 0.3981, g_loss: 2.236
[49/50] [49_38493] d_loss: 0.4604, g_loss: 2.653
[49/50] [49_38494] d_loss: 0.3957, g_loss: 2.608
[49/50] [49_38495] d_loss: 0.4222, g_loss: 2.929
[49/50] [49_38496] d_loss: 0.3007, g_loss: 1.802
[49/50] [49_38497] d_lo

[49/50] [49_38653] d_loss: 0.256, g_loss: 3.267
[49/50] [49_38654] d_loss: 0.1978, g_loss: 2.222
[49/50] [49_38655] d_loss: 0.4035, g_loss: 2.999
[49/50] [49_38656] d_loss: 0.2441, g_loss: 2.612
[49/50] [49_38657] d_loss: 0.59, g_loss: 1.698
[49/50] [49_38658] d_loss: 0.4584, g_loss: 1.511
[49/50] [49_38659] d_loss: 0.5088, g_loss: 2.12
[49/50] [49_38660] d_loss: 0.5093, g_loss: 2.53
[49/50] [49_38661] d_loss: 0.4259, g_loss: 2.172
[49/50] [49_38662] d_loss: 0.4684, g_loss: 2.487
[49/50] [49_38663] d_loss: 0.4976, g_loss: 1.398
[49/50] [49_38664] d_loss: 0.3219, g_loss: 1.783
[49/50] [49_38665] d_loss: 0.2652, g_loss: 2.355
[49/50] [49_38666] d_loss: 0.3855, g_loss: 1.706
[49/50] [49_38667] d_loss: 0.5042, g_loss: 1.58
[49/50] [49_38668] d_loss: 0.5081, g_loss: 1.486
[49/50] [49_38669] d_loss: 0.308, g_loss: 2.675
[49/50] [49_38670] d_loss: 0.1842, g_loss: 2.223
[49/50] [49_38671] d_loss: 0.3104, g_loss: 1.919
[49/50] [49_38672] d_loss: 0.4212, g_loss: 3.105
[49/50] [49_38673] d_loss: 

[49/50] [49_38831] d_loss: 0.5578, g_loss: 1.273
[49/50] [49_38832] d_loss: 0.6979, g_loss: 1.669
[49/50] [49_38833] d_loss: 0.5545, g_loss: 1.779
[49/50] [49_38834] d_loss: 0.3722, g_loss: 2.443
[49/50] [49_38835] d_loss: 0.2965, g_loss: 2.072
[49/50] [49_38836] d_loss: 0.5751, g_loss: 1.873
[49/50] [49_38837] d_loss: 0.4739, g_loss: 2.049
[49/50] [49_38838] d_loss: 0.2021, g_loss: 2.982
[49/50] [49_38839] d_loss: 0.4281, g_loss: 1.787
[49/50] [49_38840] d_loss: 0.6284, g_loss: 1.676
[49/50] [49_38841] d_loss: 0.6614, g_loss: 1.915
[49/50] [49_38842] d_loss: 0.4032, g_loss: 2.569
[49/50] [49_38843] d_loss: 0.4004, g_loss: 1.501
[49/50] [49_38844] d_loss: 0.3218, g_loss: 3.544
[49/50] [49_38845] d_loss: 0.3181, g_loss: 2.199
[49/50] [49_38846] d_loss: 0.2598, g_loss: 2.788
[49/50] [49_38847] d_loss: 0.4388, g_loss: 1.814
[49/50] [49_38848] d_loss: 0.2915, g_loss: 3.206
[49/50] [49_38849] d_loss: 0.4435, g_loss: 0.9507
[49/50] [49_38850] d_loss: 0.8391, g_loss: 2.914
[49/50] [49_38851] 

[49/50] [49_39001] d_loss: 0.3828, g_loss: 1.716
[49/50] [49_39002] d_loss: 0.2715, g_loss: 3.141
[49/50] [49_39003] d_loss: 0.7184, g_loss: 1.63
[49/50] [49_39004] d_loss: 0.4456, g_loss: 2.565
[49/50] [49_39005] d_loss: 0.2236, g_loss: 2.251
[49/50] [49_39006] d_loss: 0.3777, g_loss: 2.095
[49/50] [49_39007] d_loss: 0.3976, g_loss: 2.544
[49/50] [49_39008] d_loss: 0.448, g_loss: 2.625
[49/50] [49_39009] d_loss: 0.4627, g_loss: 1.436
[49/50] [49_39010] d_loss: 0.6442, g_loss: 2.021
[49/50] [49_39011] d_loss: 0.4388, g_loss: 2.054
[49/50] [49_39012] d_loss: 0.4236, g_loss: 1.817
[49/50] [49_39013] d_loss: 0.3003, g_loss: 2.439
[49/50] [49_39014] d_loss: 0.5103, g_loss: 1.217
[49/50] [49_39015] d_loss: 0.4789, g_loss: 2.422
[49/50] [49_39016] d_loss: 0.2809, g_loss: 2.65
[49/50] [49_39017] d_loss: 0.2985, g_loss: 2.57
[49/50] [49_39018] d_loss: 0.3359, g_loss: 2.003
[49/50] [49_39019] d_loss: 0.5941, g_loss: 2.816
[49/50] [49_39020] d_loss: 0.5863, g_loss: 2.102
[49/50] [49_39021] d_los